In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import math

# raw 데이터 조작

In [3]:
df = pd.read_pickle('./dataset/data1.pkl')
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000 entries, 0 to 9999
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   date      10000 non-null  datetime64[ns]
 1   time      10000 non-null  object        
 2   cusno     10000 non-null  int32         
 3   gender    10000 non-null  object        
 4   age       10000 non-null  object        
 5   region    10000 non-null  object        
 6   product_  10000 non-null  int64         
 7   price     10000 non-null  int32         
 8   quantity  10000 non-null  int32         
 9   amount    10000 non-null  int32         
dtypes: datetime64[ns](1), int32(4), int64(1), object(4)
memory usage: 703.1+ KB


,date,time,cusno,gender,age,region,product_,price,quantity,amount
orderno,,,,,,,,,,
0,2020-01-01,01:21:44,555,1,40,1,20,200,15,3000
1,2020-01-01,01:31:06,505,1,40,0,20,119,10,1190
2,2020-01-01,01:56:52,736,1,40,2,20,177,9,1593
3,2020-01-01,06:04:44,445,1,50,1,10,243,5,1215
4,2020-01-01,06:20:57,72,0,30,0,10,201,1,201


## 반영할 인사이트

### 사용자 정의 함수

In [4]:
def decrease_by_for(by, for_): # for_ % 만큼의 값들에게 by %만큼감소 적용
    if np.random.randint(100) < for_: 
        return 1.0
    else:
        mul = ((100-by)/100) + np.random.uniform(0.0, 0.1)
        return round(mul, 3)

In [5]:
def increase_by_for(by, for_): # for_ % 만큼의 값들에게 by %만큼증가 적용
    if np.random.randint(100) < for_: 
        return 1.0
    else:
        mul = ((100+by)/100) + np.random.uniform(0.0, 0.1)
        return round(mul, 3)

### 남성, 30-50대, 고가 -> 매출 증가

- 조작 전 매출 확인

In [6]:
df1 = df.copy()

print(f'*****columns = {df1.columns}')

print('\n//**********info**********//')
df1.info()
print('\n')

df1.rename({'product':'product_'}, axis=1, inplace=True)

condition1 = (df1.gender == 0)&((df1.age >= 30)&(df1.age < 60))&(df1.product_ == 10 )
print('*****1만개 중 ', condition1.sum(), '개 해당')

print('\n//**********조작전**********//')
print('*****2020년 매출', round(df1.amount[condition1 & (df1.date.dt.year == 2020)].sum()/10**4, 2), '만원')
print('*****2021년 매출', round(df1.amount[condition1 & (df1.date.dt.year == 2021)].sum()/10**4, 2), '만원')

print('\n*****2020년 물건 갯수', round(df1.quantity[condition1 & (df1.date.dt.year == 2020)].sum()/10**3, 1), ' 천개')
print('*****2021년 물건 갯수', round(df1.quantity[condition1 & (df1.date.dt.year == 2021)].sum()/10**3, 1), ' 천개')

*****columns = Index(['date', 'time', 'cusno', 'gender', 'age', 'region', 'product_', 'price',
       'quantity', 'amount'],
      dtype='object')

//**********info**********//
<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000 entries, 0 to 9999
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   date      10000 non-null  datetime64[ns]
 1   time      10000 non-null  object        
 2   cusno     10000 non-null  int32         
 3   gender    10000 non-null  object        
 4   age       10000 non-null  object        
 5   region    10000 non-null  object        
 6   product_  10000 non-null  int64         
 7   price     10000 non-null  int32         
 8   quantity  10000 non-null  int32         
 9   amount    10000 non-null  int32         
dtypes: datetime64[ns](1), int32(4), int64(1), object(4)
memory usage: 703.1+ KB


*****1만개 중  745 개 해당

//**********조작전**********//
*****2020년 매출 101.59 만원
*****2

- 조작 하기
- quantity, amount 수정

In [7]:
sr_q = df1.quantity[condition1 & (df1.date.dt.year == 2021)]

sr_q = sr_q.apply(lambda x : x*increase_by_for(50, 50))

df1.quantity[condition1 & (df1.date.dt.year == 2021)] = sr_q.apply(lambda x : math.ceil(x))

df1.quantity.astype('int')
print()

In [8]:
df1.amount = df1.price*df1.quantity

In [9]:
print('\n//**********조작후**********//')

print('*****2020년 매출', round(df1.amount[condition1 & (df1.date.dt.year == 2020)].sum()/10**4, 2), '만원')
print('*****2021년 매출', round(df1.amount[condition1 & (df1.date.dt.year == 2021)].sum()/10**4, 2), '만원')

print('\n*****2020년 물건 갯수', round(df1.quantity[condition1 & (df1.date.dt.year == 2020)].sum()/10**3, 1), '천개')
print('*****2021년 물건 갯수', round(df1.quantity[condition1 & (df1.date.dt.year == 2021)].sum()/10**3, 1), '천개')


//**********조작후**********//
*****2020년 매출 101.59 만원
*****2021년 매출 122.12 만원

*****2020년 물건 갯수 4.1 천개
*****2021년 물건 갯수 4.9 천개


- 50%에 대한 50%니까 25% 증가

### 여성, 20-40대, 저가 -> 매출 증가

- 조작 전 매출 확인

In [10]:
df2 = df1.copy()

print(f'*****columns = {df2.columns}')

print('\n//**********info**********//')
df2.info()
print('\n')

df2.rename({'product':'product_'}, axis=1, inplace=True)

condition2 = (df2.gender == 1)&((df2.age >= 20)&(df2.age < 50))&(df2.product_ == 30 )
print('*****1만개 중 ', condition2.sum(), '개 해당')

print('\n//**********조작전**********//')
print('*****2020년 매출', round(df2.amount[condition2 & (df2.date.dt.year == 2020)].sum()/10**4, 2), '만원')
print('*****2021년 매출', round(df2.amount[condition2 & (df2.date.dt.year == 2021)].sum()/10**4, 2), '만원')

print('\n*****2020년 물건 갯수', round(df2.quantity[condition2 & (df2.date.dt.year == 2020)].sum()/10**3, 1), '천개')
print('*****2021년 물건 갯수', round(df2.quantity[condition2 & (df2.date.dt.year == 2021)].sum()/10**3, 1), '천개')

*****columns = Index(['date', 'time', 'cusno', 'gender', 'age', 'region', 'product_', 'price',
       'quantity', 'amount'],
      dtype='object')

//**********info**********//
<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000 entries, 0 to 9999
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   date      10000 non-null  datetime64[ns]
 1   time      10000 non-null  object        
 2   cusno     10000 non-null  int32         
 3   gender    10000 non-null  object        
 4   age       10000 non-null  object        
 5   region    10000 non-null  object        
 6   product_  10000 non-null  int64         
 7   price     10000 non-null  int32         
 8   quantity  10000 non-null  int32         
 9   amount    10000 non-null  int32         
dtypes: datetime64[ns](1), int32(4), int64(1), object(4)
memory usage: 961.2+ KB


*****1만개 중  1524 개 해당

//**********조작전**********//
*****2020년 매출 43.44 만원
*****2

- 조작 하기
- quantity, amount 수정

In [11]:
sr_q = df2.quantity[condition2 & (df2.date.dt.year == 2021)]

sr_q = sr_q.apply(lambda x : x*increase_by_for(50, 50))

df2.quantity[condition2 & (df2.date.dt.year == 2021)] = sr_q.apply(lambda x : math.ceil(x))

df2.quantity.astype('int')
print()

In [12]:
df2.amount = df2.price*df2.quantity

In [13]:
print('\n//**********조작후**********//')

print('*****2020년 매출', round(df2.amount[condition2 & (df2.date.dt.year == 2020)].sum()/10**4, 2), '만원')
print('*****2021년 매출', round(df2.amount[condition2 & (df2.date.dt.year == 2021)].sum()/10**4, 2), '만원')

print('\n*****2020년 물건 갯수', round(df2.quantity[condition2 & (df2.date.dt.year == 2020)].sum()/10**3, 1), '천개')
print('*****2021년 물건 갯수', round(df2.quantity[condition2 & (df2.date.dt.year == 2021)].sum()/10**3, 1), '천개')


//**********조작후**********//
*****2020년 매출 43.44 만원
*****2021년 매출 51.62 만원

*****2020년 물건 갯수 8.3 천개
*****2021년 물건 갯수 10.4 천개


### 여성, 20-40대, 중&고가 -> 매출 감소

In [14]:
df3 = df2.copy()

print(f'*****columns = {df3.columns}')

print('\n//**********info**********//')
df3.info()
print('\n')

df3.rename({'product':'product_'}, axis=1, inplace=True)

condition3 = (df3.gender == 1)&((df3.age >= 20)&(df3.age < 50))&((df3.product_ == 10 ) | (df3.product_ == 20 ))
print('*****1만개 중 ', condition3.sum(), '개 해당')

print('\n//**********조작전**********//')
print('*****2020년 매출', round(df3.amount[condition3 & (df3.date.dt.year == 2020)].sum()/10**4, 2), '만원')
print('*****2021년 매출', round(df3.amount[condition3 & (df3.date.dt.year == 2021)].sum()/10**4, 2), '만원')

print('\n*****2020년 물건 갯수', round(df3.quantity[condition3 & (df3.date.dt.year == 2020)].sum()/10**3, 1), '천개')
print('*****2021년 물건 갯수', round(df3.quantity[condition3 & (df3.date.dt.year == 2021)].sum()/10**3, 1), '천개')

*****columns = Index(['date', 'time', 'cusno', 'gender', 'age', 'region', 'product_', 'price',
       'quantity', 'amount'],
      dtype='object')

//**********info**********//
<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000 entries, 0 to 9999
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   date      10000 non-null  datetime64[ns]
 1   time      10000 non-null  object        
 2   cusno     10000 non-null  int32         
 3   gender    10000 non-null  object        
 4   age       10000 non-null  object        
 5   region    10000 non-null  object        
 6   product_  10000 non-null  int64         
 7   price     10000 non-null  int32         
 8   quantity  10000 non-null  int32         
 9   amount    10000 non-null  int32         
dtypes: datetime64[ns](1), int32(4), int64(1), object(4)
memory usage: 961.2+ KB


*****1만개 중  2984 개 해당

//**********조작전**********//
*****2020년 매출 329.6 만원
*****2

- 조작 하기
- quantity, amount 수정

In [15]:
sr_q = df3.quantity[condition3 & (df3.date.dt.year == 2021)]

sr_q = sr_q.apply(lambda x : x*decrease_by_for(50, 80))

df3.quantity[condition3 & (df3.date.dt.year == 2021)] = sr_q.apply(lambda x : math.ceil(x))

df3.quantity.astype('int')
print()

In [16]:
df3.amount = df3.price*df3.quantity

In [17]:
print('\n//**********조작후**********//')

print('*****2020년 매출', round(df3.amount[condition3 & (df3.date.dt.year == 2020)].sum()/10**4, 2), '만원')
print('*****2021년 매출', round(df3.amount[condition3 & (df3.date.dt.year == 2021)].sum()/10**4, 2), '만원')

print('\n*****2020년 물건 갯수', round(df3.quantity[condition3 & (df3.date.dt.year == 2020)].sum()/10**3, 1), '천개')
print('*****2021년 물건 갯수', round(df3.quantity[condition3 & (df3.date.dt.year == 2021)].sum()/10**3, 1), '천개')


//**********조작후**********//
*****2020년 매출 329.6 만원
*****2021년 매출 278.57 만원

*****2020년 물건 갯수 16.4 천개
*****2021년 물건 갯수 13.9 천개


### C도시, 저가 -> 일부 고객 2021 매출 0원으로

In [18]:
df3.region.unique()

array([1, 0, 2], dtype=object)

In [19]:
df3.head()

,date,time,cusno,gender,age,region,product_,price,quantity,amount
orderno,,,,,,,,,,
0,2020-01-01,01:21:44,555,1,40,1,20,200,15,3000
1,2020-01-01,01:31:06,505,1,40,0,20,119,10,1190
2,2020-01-01,01:56:52,736,1,40,2,20,177,9,1593
3,2020-01-01,06:04:44,445,1,50,1,10,243,5,1215
4,2020-01-01,06:20:57,72,0,30,0,10,201,1,201


In [20]:
df4 = df3.copy()

print(f'*****columns = {df4.columns}')

print('\n//**********info**********//')
df4.info()
print('\n')

df4.rename({'product':'product_'}, axis=1, inplace=True)

condition4 = (df4.region == 2)&(df4.product_ == 10 )
print('*****1만개 중 ', condition4.sum(), '개 해당')

print('\n//**********조작전**********//')
print('*****2020년 매출', round(df4.amount[condition4 & (df4.date.dt.year == 2020)].sum()/10**4, 2), '만원')
print('*****2021년 매출', round(df4.amount[condition4 & (df4.date.dt.year == 2021)].sum()/10**4, 2), '만원')

print('\n*****2020년 물건 갯수', round(df4.quantity[condition4 & (df4.date.dt.year == 2020)].sum()/10**3, 1), '천개')
print('*****2021년 물건 갯수', round(df4.quantity[condition4 & (df4.date.dt.year == 2021)].sum()/10**3, 1), '천개')

*****columns = Index(['date', 'time', 'cusno', 'gender', 'age', 'region', 'product_', 'price',
       'quantity', 'amount'],
      dtype='object')

//**********info**********//
<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000 entries, 0 to 9999
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   date      10000 non-null  datetime64[ns]
 1   time      10000 non-null  object        
 2   cusno     10000 non-null  int32         
 3   gender    10000 non-null  object        
 4   age       10000 non-null  object        
 5   region    10000 non-null  object        
 6   product_  10000 non-null  int64         
 7   price     10000 non-null  int32         
 8   quantity  10000 non-null  int32         
 9   amount    10000 non-null  int32         
dtypes: datetime64[ns](1), int32(4), int64(1), object(4)
memory usage: 961.2+ KB


*****1만개 중  666 개 해당

//**********조작전**********//
*****2020년 매출 80.2 만원
*****202

- 조작 하기
- quantity, amount 수정

#### C도시 조작 함수

In [21]:
def delete_and_decrease(del_for, dec_by, dec_for):
    if np.random.randint(100) < del_for: 
        return 0.0
    else:
        if np.random.randint(100) < dec_for:
            return 1.0
        else:
            mul = ((100-dec_by)/100) + np.random.uniform(0.0, 0.1)
            return round(mul, 3)

In [22]:
sr_q = df4.quantity[condition4 & (df4.date.dt.year == 2021)]

sr_q = sr_q.apply(lambda x : x*delete_and_decrease(50, 0, 0))

df4.quantity[condition4 & (df4.date.dt.year == 2021)] = sr_q.apply(lambda x : math.ceil(x))

df4.quantity.astype('int')
print()

In [23]:
df4.amount = df4.price*df4.quantity

In [24]:
print('\n//**********조작후**********//')

print('*****2020년 매출', round(df4.amount[condition4 & (df4.date.dt.year == 2020)].sum()/10**4, 2), '만원')
print('*****2021년 매출', round(df4.amount[condition4 & (df4.date.dt.year == 2021)].sum()/10**4, 2), '만원')

print('\n*****2020년 물건 갯수', round(df4.quantity[condition4 & (df4.date.dt.year == 2020)].sum()/10**3, 1), '천개')
print('*****2021년 물건 갯수', round(df4.quantity[condition4 & (df4.date.dt.year == 2021)].sum()/10**3, 1), '천개')


//**********조작후**********//
*****2020년 매출 80.2 만원
*****2021년 매출 54.43 만원

*****2020년 물건 갯수 3.2 천개
*****2021년 물건 갯수 2.1 천개


#### 새 함수가 잘 적용되었는지 확인

In [25]:
df4_1 = df4.copy()
df4_1.head()

,date,time,cusno,gender,age,region,product_,price,quantity,amount
orderno,,,,,,,,,,
0,2020-01-01,01:21:44,555,1,40,1,20,200,15,3000
1,2020-01-01,01:31:06,505,1,40,0,20,119,10,1190
2,2020-01-01,01:56:52,736,1,40,2,20,177,9,1593
3,2020-01-01,06:04:44,445,1,50,1,10,243,5,1215
4,2020-01-01,06:20:57,72,0,30,0,10,201,1,201


In [26]:
df4_1[condition4 & (df4.date.dt.year == 2021)].quantity.count(), '조건에 해당되는 행의 전체 갯수'

(354, '조건에 해당되는 행의 전체 갯수')

In [27]:
df4_1[condition4 & (df4.date.dt.year == 2021) & (df4_1.quantity == 0)].quantity.count(), '0이 들어간 행의 갯수'

(176, '0이 들어간 행의 갯수')

In [28]:
df4_1.quantity.replace(0, np.nan, inplace=True)
df4_1.quantity.isna().sum(), 'nan이 들어간 행의 갯수'

(176, 'nan이 들어간 행의 갯수')

In [29]:
df4_2 = df4_1.dropna()

In [30]:
df4_2.quantity.count(), '남은 행'

(9824, '남은 행')

In [31]:
df4_2.head(10)

,date,time,cusno,gender,age,region,product_,price,quantity,amount
orderno,,,,,,,,,,
0,2020-01-01,01:21:44,555,1,40,1,20,200,15.0,3000
1,2020-01-01,01:31:06,505,1,40,0,20,119,10.0,1190
2,2020-01-01,01:56:52,736,1,40,2,20,177,9.0,1593
3,2020-01-01,06:04:44,445,1,50,1,10,243,5.0,1215
4,2020-01-01,06:20:57,72,0,30,0,10,201,1.0,201
5,2020-01-01,06:29:55,246,0,50,0,30,78,7.0,546
6,2020-01-01,08:26:29,44,0,20,1,30,14,6.0,84
7,2020-01-01,09:58:33,116,0,50,1,10,227,13.0,2951
8,2020-01-01,11:31:00,72,0,30,0,30,85,5.0,425
